# 7. AMES HOUSING: FEATURE SELECTION
---

In this chapter, we'll explore how to use correlation between features and the target column, correlation between features, and variance of features to select features. We'll specifically focus on selecting from feature columns that don't have any missing values or don't need to be transformed to be useful

## 1. Introducing the Data

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
pd.set_option("display.max_columns", 99)
pd.set_option("display.max_rows", 999)
pd.set_option('precision', 3)

ames = pd.read_csv('data/Ames_Housing.txt', delimiter="\t")

train, test = train_test_split(ames, test_size=0.2, random_state=42)
print(train.shape, test.shape)

X_train = train.drop('SalePrice', axis=1)
y_train = train['SalePrice']

X_test = test.drop('SalePrice', axis=1)
y_test = test['SalePrice']

X_train.shape, y_train.shape, X_test.shape, y_test.shape

(2344, 82) (586, 82)


((2344, 81), (2344,), (586, 81), (586,))

## 2. Missing Values

In [7]:
train.dtypes.value_counts()

object     43
int64      28
float64    11
dtype: int64

In [13]:
train_num = train.select_dtypes(include=['int64', 'float64'])
num_drop = ['PID', 'Year Built', 'Year Remod/Add', 'Garage Yr Blt', 
            'Mo Sold', 'Yr Sold']
train_num = train_num.drop(num_drop, axis=1)
is_null = train_num.isnull().sum()
full_cols = is_null[is_null == 0]
print(len(full_cols))
full_cols

23


Order              0
MS SubClass        0
Lot Area           0
Overall Qual       0
Overall Cond       0
1st Flr SF         0
2nd Flr SF         0
Low Qual Fin SF    0
Gr Liv Area        0
Full Bath          0
Half Bath          0
Bedroom AbvGr      0
Kitchen AbvGr      0
TotRms AbvGrd      0
Fireplaces         0
Wood Deck SF       0
Open Porch SF      0
Enclosed Porch     0
3Ssn Porch         0
Screen Porch       0
Pool Area          0
Misc Val           0
SalePrice          0
dtype: int64

KeyError: "None of [Int64Index([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,\n            0],\n           dtype='int64')] are in the [columns]"